Q1) Prelimanary overview =  the two datasets contain involve analysis and effect of various objective which are treated as features/variables for modelling purposes on the quality of wine for two different classes of wine. The last column contains the assigned quality to the given wine sample based on various features which we can treat as target values for regression problem. Exact quantification can be found in attached documentaion which validates its use for scienctific purposes especially regression and feature selection

1 b) Explore = invloves identifying feautures , target values , whether their are any null points or not, identify outliers

Visulaise = maybe print few rows (chat gpt recommendation), quantify no. of null points, outliers
print some basic statistcs like mean median mode

Preprocessing = fill null points using strategies like imputation(gpt), removing outliers
chat gpt := which may include feature scaling, normalization, or encoding categorical variables.

In [2]:
import pandas as pd
import types
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [5]:
##explore and visualize

# Load the dataset
wine_red = pd.read_csv("winequality-red.csv", sep=';')
wine_white = pd.read_csv("winequality-white.csv", sep=';')

##deeisplay various parameters asosciated with data
##convection input matrix = N x D

##no. of rows = no of samples
N_red = wine_red.shape[0] ##axis 0
N_white = wine_white.shape[0] ##axis 0
##no of features= no of columns - 1(target column)
D_red = wine_red.shape[1] - 1 ##axis 1
D_white = wine_white.shape[1] - 1 ##axis 1 

##seperate features and target
featuresRed = wine_red.drop('quality', axis=1)##quality = target selectds all columns expect quality
targetRed = wine_red['quality']
featuresWhite = wine_white.drop('quality', axis=1)
targetWhite = wine_white['quality']

##desriptive stats assosciated with data ref : https://www.w3schools.com/python/pandas/ref_df_describe.asp 
Des_Red = wine_red.describe() ##obj connecting description (mean , median mode, max etc of data)
Des_white = wine_white.describe()

##null points
##ref chat gpt ,is_null converts every datapoint to bool (Non empty => 0 empty => non zero)
NullCountRed = wine_red.isnull().sum() ##sum all non zero gives null count in every column
NullCountWhite = wine_white.isnull().sum()

##outliers bas
##ref : chatgpt IQR method basically calculate data range within which 50% of middle data points
def count_outliers(df, columnName):
    Q1 = df[columnName].quantile(0.25) # first quartile of data
    Q3 = df[columnName].quantile(0.75)# rnd quartile 
    IQR = Q3 - Q1  #3 range we are assuming to contain data (excluding outliers)
    lower_bound = Q1 - 1.5 * IQR ##chat gpt selected appropriate limits below which we can easily say data is outlier
    upper_bound = Q3 + 1.5 * IQR
     ##chat gpt syntax
     ##only selects that part of selected column of dataframe which lies in limits
    df_removedOutliers = (df[columnName] >= lower_bound) & (df[columnName] <= upper_bound)
    OutliersCount = df_removedOutliers.shape - df_removedOutliers.sum() ##only true i.e 1 counted in sum
    return OutliersCount




In [6]:
##visulaize the exploration above for red data same will be for white
outNumber = count_outliers(wine_red,"quality")

print(f"""The given data has {N_red} samples\n  
{D_red}  no. of features\n 
{outNumber[0]} no of ouliers in target set\n  
{NullCountRed} no. of null points in each feature and target set\n
description as follow {Des_Red}""")

The given data has 1599 samples
  
11  no. of features
 
28 no of ouliers in target set
  
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 no. of null points in each feature and target set

description as follow        fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.

In [3]:
## pre processing

##eliminate outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25) # first quartile of data
    Q3 = df[column].quantile(0.75)# rnd quartile 
    IQR = Q3 - Q1  #3 range we are assuming to contain data (excluding outliers)
    lower_bound = Q1 - 1.5 * IQR ##chat gpt selected appropriate limits below which we can easily say data is outlier
    upper_bound = Q3 + 1.5 * IQR
    ##replace outliers with more appropriate values say median
    median = df[column].median()
    ##chat gpt syntax
    ##basically uses lambda function to check if given data point is an outlier and if yes then replace with median
    df[column] = df[column].apply(lambda x: median_value if x < lower_bound or x > upper_bound else x)
    return df

##fill null points
##ref :  https://scikit-learn.org/stable/modules/impute.html from Q2 and chat gpt
def impute_data(data):
    ##select feautres to impute
    features_for_imputation = data.columns.tolist()
    # Initialize IterativeImputer
    imputer = IterativeImputer(random_state=0)
    # Fit and transform the data using IterativeImputer chat gpt 
    data_imputed = pd.DataFrame(imputer.fit_transform(data[features_for_imputation]), columns=features_for_imputation)
    ##transformed data
    data[features_for_imputation] = data_imputed


##split data
##ref : https://www.geeksforgeeks.org/how-to-split-the-dataset-with-scikit-learns-train_test_split-function/
def trainTestSplit(X,y):
    ##X = FEATURE AND Y =TARGET
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) ##70 % for training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return [[X_train, X_test, X_val,y_train], [y_test,  y_val]]

##standardize values
def standardize(X_train, X_val, X_test):
    # Standardize features to normalize independent features might be having different scales/units
    #3 ref of code snippet : https://www.digitalocean.com/community/tutorials/standardscaler-function-in-python
    scaler = StandardScaler()
    ##scale training , testing and validation data 
    X_train_scaled = scaler.fit_transform(X_train) 
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, X_val_scaled
    




1 b) lets make a class for stuff done yet for this part which we can use again again across questions
Can be directly used in comprehensive python script

In [4]:
class ProcessedData:
    def __init__(self,data):
        self.data = data
        self.N = data.shape[0] ##axis 0
        self.D = data.shape[1] - 1 ##axis 1 
        self.features = data.drop('quality', axis=1)##quality = target selectds all columns expect quality
        self.target = data['quality']
        self.description = data.describe()
        self.NullCount= data.isnull().sum()
        
    def count_outliers(self, columnName):
        df = self.data
        Q1 = df[columnName].quantile(0.25) # first quartile of data
        Q3 = df[columnName].quantile(0.75)# rnd quartile 
        IQR = Q3 - Q1  #3 range we are assuming to contain data (excluding outliers)
        lower_bound = Q1 - 1.5 * IQR ##chat gpt selected appropriate limits below which we can easily say data is outlier
        upper_bound = Q3 + 1.5 * IQR
        df_removedOutliers = (df[columnName] >= lower_bound) & (df[columnName] <= upper_bound)
        self.OutliersCount = df_removedOutliers.shape - df_removedOutliers.sum() ##only true i.e 1 counted in sum
        return OutliersCount
    
    def remove_outliers(self, column):
        df = self.data
        Q1 = df[column].quantile(0.25) # first quartile of data
        Q3 = df[column].quantile(0.75)# rnd quartile 
        IQR = Q3 - Q1  #3 range we are assuming to contain data (excluding outliers)
        lower_bound = Q1 - 1.5 * IQR ##chat gpt selected appropriate limits below which we can easily say data is outlier
        upper_bound = Q3 + 1.5 * IQR
        median = df[column].median()
        df[column] = df[column].apply(lambda x: median if x < lower_bound or x > upper_bound else x)
        self.data = df
        
    def impute_data(self):
        ##select feautres to impute
        features_for_imputation = self.data.columns.tolist()
        # Initialize IterativeImputer
        imputer = IterativeImputer(random_state=0)
        # Fit and transform the data using IterativeImputer ref : chat gpt 
        data_imputed = pd.DataFrame(imputer.fit_transform(self.data[features_for_imputation]), columns=features_for_imputation)
        ##transformed data
        self.data[features_for_imputation] = data_imputed

    def trainTestSplit(self):
        ##X = FEATURE AND Y =TARGET, of new proccedData which is stored in self.data
        X = self.data.drop('quality', axis=1)
        y = self.data["quality"]
        self.X_train, X_temp, self.y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) ##70 % for training
        self.X_val, self.X_test, self.y_val, self.y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    def standardize(self):
        scaler = StandardScaler()
        self.X_train_scaled = scaler.fit_transform(self.X_train) 
        self.X_val_scaled = scaler.transform(self.X_val)
        self.X_test_scaled = scaler.transform(self.X_test)
    

1 c) Train, validate varying at least one hyperparameter, and test at least two types of models:
i. Random forest
ii. Support vector regression with RBF kernel
iii. Neural network with single hidden layer (output layer should have linear activation)

In [5]:
## lets make a class for each model which can be used repeatedely

class model:
    ##model for a given dataset
    def __init__(self,ProcessedData): ##pass object of type ProcessedData (have all member functions,variables)
        self.data = ProcessedData
        ##Processed data must have been splited in testing and training set a priori
        self.X_train_scaled = ProcessedData.X_train_scaled  ##we need training data for this class
        self.y_train = ProcessedData.y_train
    # 1. Random Forest  (creates multiple decesion tress and average out the prediction value of each tree)
    ##ref chat gpt and  scikit_learn RandomForestRegressor
    
    def randomForest(self): 
        rf_model = RandomForestRegressor(random_state=42) ## initialize with initial random value 42(chat gpt) reproducibility
        Paramgrid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10]}##hyperparameters varying = no of trees, depth of trees
        ##use GridSearchCV to minimize mean square error , and find best hyperparams to fit model on training data
        grid_rf = GridSearchCV(rf_model, Paramgrid_rf, cv=3, scoring='neg_mean_squared_error')## 3 fold verification
        grid_rf.fit(self.X_train_scaled, self.y_train)##fit model
        n_estimators, max_depth =  grid_rf.best_params_ #3optimum hyperparams
        ##corresponding best model
        Best_rf_model = grid_rf.best_estimator_
        return Best_rf_model
    
    ##2. SVR using rbf kernel    
    ##ref chat gpt and  scikit_learn SVR
    def SVR(self):   
        svr_model = SVR(kernel='rbf') ##initialize with rbf kernel
        Paramgrid_svr = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}##hyperparams varying = regularization coefficient(prevents overfitting) and kernel coefficient for rbf
        ##use GridSearchCV to minimize mean square error , and find best hyperparams to fit model on training data
        grid_svr = GridSearchCV(svr_model, Paramgrid_svr, cv=3, scoring='neg_mean_squared_error')##search on grid to find best hyperparams
        grid_svr.fit(self.X_train_scaled, self.y_train) ##3 fit modekl
        ##optimum c and gamma
        c, gamma =  grid_svr.best_params_
        ##return best model
        return grid_svr.best_estimator_
    
    ##neural networks
    ## ref https://www.analyticsvidhya.com/blog/2021/11/training-neural-network-with-keras-and-basics-of-deep-learning/
    ## simple feedforward neural network with 1 hidden layer
    def CreateNeuralNetwork(self):
        nn_model = Sequential() ##linear stack of layers in keras
        ## 10 neurons per layer, relu activation functiom
        nn_model.add(Dense(10, input_dim=self.X_train_scaled.shape[1], activation='relu'))
        ##output layer with linear activation as asked in Ques
        nn_model.add(Dense(1, activation='linear')) # linear activation
        ##loss = mean sq error for regression, call adams optimizer for keras
        nn_model.compile(optimizer='adam', loss='mean_squared_error') #minimize mean square error
        return nn_model
    
    ##now using kerras generate a nn model we can use
    def NN(self):
        nn_model = KerasRegressor(build_fn=self.CreateNeuralNetwork, epochs=50, batch_size=32, verbose=0)
        ##finally fir model with training data
        nn_model.fit(self.X_train_scaled, self.y_train)
        return nn_model
        

For subsequent question from now on we can directly use these classes for processing data and deploying various models  

In [8]:
## Q1 c using the classes we defined above
# Load the dataset
wine_red = pd.read_csv("winequality-red.csv", sep=';')
wine_white = pd.read_csv("winequality-white.csv", sep=';')
## create instance of class ProcessData
Red = ProcessedData(wine_red)
White = ProcessedData(wine_white)
##remove outliers in target
Red.remove_outliers("quality")
White.remove_outliers("quality")
##fill null points
Red.impute_data()
White.impute_data()
##split 
Red.trainTestSplit()
White.trainTestSplit()
##finally standardise
Red.standardize()
White.standardize()

##now our datasets have been processed and splitted into training test and validation 
##train the three models
ModelRed = model(Red) ##instance of model class with different data
ModelWhite = model(White)

# Validate Random Forest model
rfRed = ModelRed.randomForest()
RfpredictionsRed = rfRed.predict(Red.X_val_scaled) ##model predictions on validation data
rf_mseRed = mean_squared_error(Red.y_val, RfpredictionsRed) ##compute mean square error\
print("Random Forest Validation MSE:",rf_mseRed)
##for white same old same old
rfWhite = ModelWhite.randomForest()
RfpredictionsWhite = rfWhite.predict(White.X_val_scaled) ##model predictions on validation data
rf_mseWhite = mean_squared_error(White.y_val, RfpredictionsWhite) ##compute mean square error

## Validate SVR model
svrRed = ModelRed.SVR() ##created model
svr_predictionsRed = svrRed.predict(Red.X_val_scaled) ##made predictions
svr_mseRed = mean_squared_error(Red.y_val, svr_predictionsRed)##calculated error
print("SVR Validation MSE:", svr_mseRed)
#same for white
svrWhite = ModelWhite.SVR()
svr_predictionsWhite = svrWhite.predict(White.X_val_scaled)
svr_mseWhite = mean_squared_error(White.y_val, svr_predictionsWhite)
print("SVR Validation MSE:", svr_mseWhite)

# Validate Neural Network model
nn_modelRed = ModelRed.NN() ##initialized model
nn_predictionsRed = nn_modelRed.predict(Red.X_val_scaled) ##predict
nn_mseRed = mean_squared_error(Red.y_val, nn_predictionsRed)## cal error
print("Neural Network Validation MSE:", nn_mseRed)
##same for white
nn_modelWhite = ModelWhite.NN() ##initialized model
nn_predictionsWhite = nn_modelWhite.predict(White.X_val_scaled) ##predict
nn_mseWhite = mean_squared_error(White.y_val, nn_predictionsRed)## cal error
print("Neural Network Validation MSE:", nn_mseWhite)

# Validate Neural Network model
nn_modelRed = ModelRed.NN() ##initialized model
nn_predictionsRed = nn_modelRed.predict(Red.X_val_scaled) ##predict
nn_mseRed = mean_squared_error(Red.y_val, nn_predictionsRed)## cal error
print("Neural Network Validation MSE:", nn_mseRed)
##same for white
nn_modelWhite = ModelWhite.NN() ##initialized model
nn_predictionsWhite = nn_modelWhite.predict(White.X_val_scaled) ##predict
nn_mseWhite = mean_squared_error(White.y_val, nn_predictionsWhite)## cal error
print("Neural Network Validation MSE:", nn_mseWhite)

Random Forest Validation MSE: 0.3111543055555555
SVR Validation MSE: 0.3419715627591982
SVR Validation MSE: 0.38032304744574225


/tmp/ipykernel_8147/214265759.py:52: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn_model = KerasRegressor(build_fn=self.CreateNeuralNetwork, epochs=50, batch_size=32, verbose=0)


Neural Network Validation MSE: 0.8972403624263711


AttributeError: 'model' object has no attribute 'nn_model'

/tmp/ipykernel_8147/214265759.py:52: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn_model = KerasRegressor(build_fn=self.CreateNeuralNetwork, epochs=50, batch_size=32, verbose=0)


Neural Network Validation MSE: 0.8898504805946515


/tmp/ipykernel_8147/214265759.py:52: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn_model = KerasRegressor(build_fn=self.CreateNeuralNetwork, epochs=50, batch_size=32, verbose=0)


Neural Network Validation MSE: 0.42704491259493194


Q 1c) importance of features

In [ ]:
##q1 d
